In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
#read poi file
pitch_velo = pd.read_csv('pitching_poi_metrics.csv')

In [ ]:
# seperates 'good' pitches from 'bad' pitches
pitch_velo['mph_bin'] = pd.qcut(pitch_velo['pitch_speed_mph'], 
                                q=2, labels = ['bad','good'])

In [ ]:
# pulls out good pitches to see if they seperate correctly
good_mph = pitch_velo.groupby(pitch_velo['mph_bin'])
print(good_mph)

In [ ]:
# finds good pitches
good_mph_data = pitch_velo[pitch_velo['mph_bin'] == 'good']

# finds 'bad' pitches
bad_mph_data = pitch_velo[pitch_velo['mph_bin'] == 'bad']

# Specify the file paths for the Excel files
good_mph_file_path = 'good_mph_data.xlsx'
bad_mph_file_path = 'bad_mph_data.xlsx'

# Write the filtered data to separate Excel files
good_mph_data.to_excel(good_mph_file_path, index=False)
bad_mph_data.to_excel(bad_mph_file_path, index=False)

In [ ]:
# reads good and bad files
good_poi=pd.read_excel('good_mph_data.xlsx')
bad_poi=pd.read_excel('bad_mph_data.xlsx')

In [ ]:
# reads the full sig data file from driveline
JointA = pd.read_csv('joint_angles.csv')

In [ ]:
# Find common pitches from good velo to full sig data
common_users = set(JointA['session_pitch']).intersection(good_poi['session_pitch'])

# Add New Column
JointA['appears_in_pitch_velo'] = np.nan

# Populate the New Column, yes means good velo, no means 'bad'
for index, row in JointA.iterrows():
    user_value = row['session_pitch']
    if user_value in common_users:
        JointA.at[index, 'appears_in_pitch_velo'] = 'yes'
    else:
        JointA.at[index, 'appears_in_pitch_velo'] = 'no'

# Save the Updated Data

JointA.to_csv('fullsig_with_appearance.csv', index=False)

In [ ]:
# reads new file
fullsig_wa = pd.read_csv('fullsig_with_appearance.csv')

#finds length of each pitch
min_time_per_pitcher = fullsig_wa.groupby('session_pitch')['time'].min()
max_time_per_pitcher = fullsig_wa.groupby('session_pitch')['time'].max()

# Calculate Time Duration for Each Pitcher
time_duration_per_pitcher = max_time_per_pitcher - min_time_per_pitcher

# Calculate the Percentage of Time for Each Data Point and Round to the Nearest Tenth
def calculate_percentage_of_time(row):
    pitcher_id = row['session_pitch']
    time_value = row['time']
    percentage = (time_value - min_time_per_pitcher[pitcher_id]) / time_duration_per_pitcher[pitcher_id]
    return round(percentage, 2) 

# adds column with the time percentage for each frame
fullsig_wa['time_percentage'] = fullsig_wa.apply(calculate_percentage_of_time, axis=1)

# Print or Save the DataFrame with the New 'time_percentage' Column
print(fullsig_wa)

# If you want to save the DataFrame with the new column to a CSV file
fullsig_wa.to_csv('updated_pitchers_data.csv', index=False)

In [ ]:
average_good_pitch = fullsig_wa[fullsig_wa['appears_in_pitch_velo'] == 'yes'].groupby('time_percentage')['torso_pelvis_angle_z'].mean()
average_bad_pitch = fullsig_wa[fullsig_wa['appears_in_pitch_velo'] == 'no'].groupby('time_percentage')['torso_pelvis_angle_z'].mean()

# Plot the Graph
plt.figure(figsize=(8, 6))
plt.plot(average_good_pitch.index, average_good_pitch, label='Average of Good Pitches', color='green')
plt.plot(average_bad_pitch.index, average_bad_pitch, label='Average of Bad Pitches', color='red')

plt.xlabel('Time Percentage')
plt.ylabel('Average Torso Pelvis Angle')
plt.title('Average Torso Pelvis Angle in Pitching Delivery')
plt.legend()
plt.grid(True)
plt.show()